In [1]:
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from swarm_sim import *

%matplotlib inline 

In [2]:
NB_NODES = 50
DURATION = 10000   # Nb samples
REVOLUTION = 1800  # Nb samples
SAMPLE_FREQ = 0.1  # Hz, 1 sample every 10 seconds
CONNECTION_RANGE = 30000 # m

# Variables globales figures
TMAX = REVOLUTION/SAMPLE_FREQ
IDX = np.arange(0, TMAX, 1/SAMPLE_FREQ) # conversion en secondes

## 1. Formattage des données pour l'analyse
### 1.1 Importation du dataset

In [3]:
PATH = '..\..\data\cnes_swarm50\\track_'
satellites = {}

with tqdm(total=NB_NODES, desc='Extracting data') as pbar:
    for i in range(NB_NODES):
        df = pd.read_csv(PATH+str(i)+'.csv')
        df['coords'] = ['x','y','z']
        satellites[i] = df.set_index('coords', drop=True)
        pbar.update(1)

satellites[0]

Extracting data: 100%|██████████| 50/50 [00:09<00:00,  5.55it/s]


,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
coords,,,,,,,,,,,,,,,,,,,,,
x,-4.428846e+05,-4.516533e+05,-4.604007e+05,-4.691266e+05,-4.778306e+05,-4.865122e+05,-4.951711e+05,-5.038068e+05,-5.124189e+05,-5.210071e+05,...,1.440812e+06,1.444147e+06,1.447475e+06,1.450795e+06,1.454108e+06,1.457413e+06,1.460711e+06,1.464001e+06,1.467284e+06,1.470559e+06
y,-7.388676e+05,-7.522884e+05,-7.656739e+05,-7.790237e+05,-7.923370e+05,-8.056132e+05,-8.188518e+05,-8.320520e+05,-8.452134e+05,-8.583352e+05,...,2.329204e+06,2.334225e+06,2.339233e+06,2.344229e+06,2.349213e+06,2.354185e+06,2.359145e+06,2.364092e+06,2.369028e+06,2.373951e+06
z,-2.011435e+06,-2.004528e+06,-1.997528e+06,-1.990435e+06,-1.983248e+06,-1.975968e+06,-1.968596e+06,-1.961132e+06,-1.953577e+06,-1.945930e+06,...,3.661057e+06,3.655167e+06,3.649258e+06,3.643331e+06,3.637384e+06,3.631419e+06,3.625435e+06,3.619433e+06,3.613412e+06,3.607372e+06


### 1.2 Conversion en objet Swarm (module swarm_sim)

In [4]:
swarm_data = {}

with tqdm(total = REVOLUTION, desc = 'Converting to Swarm') as pbar:
    for t in range(REVOLUTION):
        swarm_data[t] = Swarm(
            connection_range=CONNECTION_RANGE, 
            nodes=[Node(id, node[str(t)].x, node[str(t)].y, node[str(t)].z) for id,node in satellites.items()]
            )
        pbar.update(1)

Converting to Swarm: 100%|██████████| 1800/1800 [00:05<00:00, 305.32it/s]


### 1.3 Création de la topologie : établissement des ISL d'après la portée de connexion

In [5]:
# Génération des matrices d'adjacence
neighbor_matrix_list = [swarm_data[t].neighbor_matrix(weighted=True) for t in range(REVOLUTION)]

In [6]:
# Création des graphes associés  
with tqdm(total=REVOLUTION, desc='Création des graphes') as pbar:
    for t in range(REVOLUTION):
        swarm_data[t].create_graph()
        pbar.update(1)

Création des graphes: 100%|██████████| 1800/1800 [00:17<00:00, 104.83it/s]


In [7]:
# Enlever les ISL trop chers de l'essaim (ceux dont le coût est supérieur au coût du plus court chemin)
with tqdm(total = REVOLUTION, desc = 'Nettoyage des ISL redondants') as pbar:
    for t in range(REVOLUTION):
        swarm_data[t].remove_expensive_edges()
        pbar.update(1)

Nettoyage des ISL redondants: 100%|██████████| 1800/1800 [06:24<00:00,  4.68it/s]


In [8]:
def rmse(data, ref=None):
    """
    This function calculates the Root Mean Square Error (RMSE) between the observed distribution and a reference value.

    Parameters:
    data (list or numpy array): A list or numpy array containing the observed data points.
    ref (float, optional): A reference value to compare the observed distribution with. Defaults to the mean of the observed data.

    Returns:
    float: The RMSE value, which represents the standard deviation of the differences between the observed data and the reference value.

    Example:
    >>> data = [1, 2, 3, 4, 5]
    >>> ref = 3
    >>> rmse(data, ref)
    0.8164965809277461
    """
    if ref is None:
        ref = np.mean(data)
    errors = [(e - ref) ** 2 for e in data]
    ratio = sum(errors) / len(data)
    return np.sqrt(ratio)

In [9]:
swarm = swarm_data[0]
strength_ref = swarm.strength()
strength_ref

[10.75,
 8.75,
 2.0,
 1.75,
 7.0,
 1.75,
 7.0,
 1.75,
 1.5,
 6.5,
 5.5,
 3.75,
 10.5,
 4.0,
 9.5,
 3.25,
 5.5,
 4.0,
 11.5,
 3.5,
 3.25,
 11.0,
 1.75,
 1.5,
 7.25,
 6.25,
 4.25,
 2.0,
 4.0,
 3.0,
 4.0,
 1.0,
 2.75,
 5.25,
 2.75,
 5.0,
 4.0,
 6.25,
 5.25,
 7.0,
 3.25,
 8.75,
 4.0,
 6.0,
 10.5,
 7.25,
 3.5,
 5.0,
 5.75,
 1.0]

In [11]:
def RND(swarm, n=10, s=1):
    """
    This function assigns each node in the swarm to a random group.

    Parameters:
    n (int, optional): The number of groups to create. Defaults to 10.
    s (int, optional): A random seed for the random group assignment. Defaults to 1.

    Returns:
    dict: A dictionary where the keys are the group IDs and the values are lists of node IDs.
    """
    groups = {}  
    for i, node in enumerate(swarm.nodes):
        node.random_group(range(n), s * i)
    for group_id in range(n):
        groups[group_id] = [node.id for node in swarm.nodes if node.group == group_id]
    return groups

In [12]:
RND(swarm)

{0: [2, 19, 31, 43],
 1: [14, 28, 32, 42, 46, 49],
 2: [1, 18, 20, 21, 22],
 3: [3, 4, 8, 15, 26, 39],
 4: [13, 23, 45],
 5: [7, 16, 36, 47],
 6: [0, 24, 25, 38, 41, 44],
 7: [9, 11, 12, 27, 40],
 8: [17, 29, 30, 34, 35, 48],
 9: [5, 6, 10, 33, 37]}

In [64]:
def MIRW(self, n=10, s=1):
    """
    This function assigns each node in the swarm to a random group by following the Multiple Independent Random Walks (MIRW) algorithm.

    Parameters:
    n (int, optional): The number of groups to create. Defaults to 10.
    s (int, optional): A random seed for the random group assignment. Defaults to 1.

    Returns:
    dict: A dictionary where the keys are the group IDs and the values are lists of node IDs.

    Example:
    >>> swarm = Swarm(...)  # Initialize a swarm object
    >>> groups = swarm.MIRW(n=10, s=1)
    >>> print(groups)
    {0: [1, 2, 3], 1: [4, 5, 6], 2: [7, 8, 9], 3: [10, 11, 12], 4: [13, 14, 15], 5: [16, 17, 18], 6: [19, 20, 21], 7: [22, 23, 24], 8: [25, 26, 27], 9: [28, 29, 30]}
    """
    sources = sample(self.nodes, n) # Initial random sources
    groups = {} # Dict(group ID:list(Node ID))
    for group_id, src in enumerate(sources): # Initialize swarms
        src.set_group(group_id)
        groups[group_id] = [src.id]
    free_nodes = [n.id for n in self.nodes if n.group==-1]
    while free_nodes: # Spread paths
        for group_id in groups.keys():
            nid1 = groups[group_id][-1] # Current node
            n1 = self.get_node_by_id(nid1)
            free_neighbors = list(set(free_nodes).intersection(n1.neighbors.keys()))
            if free_neighbors: # At least one unassigned neighbor
                nid2 = random_node(search_list=free_neighbors, s=s*group_id) # Next node
            else:
                if free_nodes == []:
                    break
                nid2 = random_node(search_list=free_nodes) # If no neighbor, perform random jump in the graph
            n2 = self.get_node_by_id(nid2)
            n2.set_group(n1.group)
            groups[group_id].append(nid2)
            free_nodes.remove(nid2)
    return groups



def random_node(search_list, s=1):
    seed(s)
    return choice(search_list)
    

In [65]:
swarm.reset_groups()
MIRW(swarm)

{0: [36, 44, 18, 21, 45],
 1: [48, 1, 37, 35, 20],
 2: [4, 33, 42, 17, 15],
 3: [16, 32, 2, 12, 0],
 4: [7, 40, 9, 11, 34],
 5: [31, 6, 47, 19, 26],
 6: [28, 29, 13, 8, 27],
 7: [30, 5, 46, 14, 39],
 8: [41, 10, 25, 3, 23],
 9: [24, 43, 38, 22, 49]}

In [70]:
def FFD(self, n=10, p=0.7, s=1):
    """
    This function assigns each node in the swarm to a group using the Forest Fire Division (FFD) algorithm.

    Parameters:
    n (int, optional): The number of groups to create. Defaults to 10.
    p (float, optional): The fire spreading probability. Defaults to 0.7.
    s (int, optional): A random seed for the random group assignment. Defaults to 1.

    Returns:
    dict: A dictionary where the keys are the group IDs and the values are lists of node IDs.
    """
    sources = sample(self.nodes, n) # Initial random sources
    groups = {} # Dict(group ID:list(Node ID))
    for group_id,src in enumerate(sources): # Initialize swarms
        src.set_group(group_id)
        groups[group_id] = [src.id]
    free_nodes = [n.id for n in self.nodes if n.group==-1]
    burning_nodes = sources
    next_nodes = []
    while free_nodes: # Spread paths from each burning node in parallel
        for bn in burning_nodes:
            if not free_nodes:
                break
            free_neighbors = list(set(free_nodes).intersection(bn.neighbors.keys()))
            if free_neighbors: # At least one unassigned neighbor
                nodes = proba_nodes(search_list=free_neighbors, proba=p, s=s) # Next node(s)
                for nid in nodes:
                    n = self.get_node_by_id(nid)
                    n.set_group(bn.group)
                    groups[bn.group].append(nid)
                    free_nodes.remove(nid)
                    next_nodes.append(n)
            else:
                nid = random_node(search_list=free_nodes) # If no neighbor, perform random jump in the graphn.set_group(bn.group)
                n = self.get_node_by_id(nid)
                n.set_group(bn.group)
                groups[bn.group].append(nid)
                free_nodes.remove(nid)
                next_nodes.append(n)
        burning_nodes = next_nodes
    return groups



def proba_nodes(search_list, proba=0.7, s=1):
    seed(s)
    trial = binomial(1, proba, len(search_list))
    nodes = [n for i,n in enumerate(search_list) if trial[i]==1] # Select the nodes that obtained a success above
    return nodes

In [84]:
swarm.reset_groups()
FFD(swarm, p=0.7)

{0: [36, 11, 44, 14, 9, 34, 18, 26, 45],
 1: [48, 32, 38, 46, 1, 8, 22, 17],
 2: [4, 6, 10, 42, 43, 25, 29, 47, 19, 35, 15, 20],
 3: [16, 37, 27],
 4: [7, 40, 23, 3, 39],
 5: [31, 5, 49],
 6: [28, 33, 13],
 7: [30, 2],
 8: [41, 0, 12],
 9: [24, 21]}